In [8]:
import cv2
import os
import numpy as np
from skimage.feature import hog
from skimage import exposure

# Define HOG parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# Define function to extract HOG features
from skimage.feature import greycomatrix, greycoprops
from skimage.filters import gabor_kernel


# Define HOG parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# Define function to extract HOG features
def extract_features(image):
    """
    Extracts HOG features from the given image
    """
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Calculate gradient images in x and y directions
    gx = cv2.Sobel(gray_image, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(gray_image, cv2.CV_32F, 0, 1)

    # Calculate magnitude and angle of gradients
    magnitude, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)

    # Create histogram of gradient orientations for each cell
    histogram = np.zeros((gray_image.shape[0] // pixels_per_cell[0],
                          gray_image.shape[1] // pixels_per_cell[1],
                          orientations))
    for i in range(histogram.shape[0]):
        for j in range(histogram.shape[1]):
            cell_magnitude = magnitude[i*pixels_per_cell[0]:(i+1)*pixels_per_cell[0],
                                       j*pixels_per_cell[1]:(j+1)*pixels_per_cell[1]]
            cell_angle = angle[i*pixels_per_cell[0]:(i+1)*pixels_per_cell[0],
                                j*pixels_per_cell[1]:(j+1)*pixels_per_cell[1]]
            hist = np.histogram(cell_angle, bins=orientations, range=(0, 180), weights=cell_magnitude)[0]
            histogram[i, j] = hist

    # Normalize blocks of cells using L1-norm
    normalized_blocks = np.zeros((histogram.shape[0] - cells_per_block[0] + 1,
                                  histogram.shape[1] - cells_per_block[1] + 1,
                                  cells_per_block[0] * cells_per_block[1] * orientations))
    for i in range(normalized_blocks.shape[0]):
        for j in range(normalized_blocks.shape[1]):
            block = histogram[i:i+cells_per_block[0], j:j+cells_per_block[1], :].flatten()
            normalized_blocks[i, j] = block / np.linalg.norm(block, ord=1)

    # Flatten feature matrix and return
    hog_features = normalized_blocks.flatten()
    return hog_features
 
   


In [9]:
# Load the dataset
X = []  # Input images
y = []  # Output labels
# Path to the directories containing the benign and malignant images

benign_path = "benign"
malignant_path = "malignant"

# Load the benign images and assign labels
for filename in os.listdir(benign_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(benign_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        X.append(image)
        y.append(0)

# Load the malignant images and assign labels
for filename in os.listdir(malignant_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(malignant_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        X.append(image)
        y.append(1)





# Split the dataset into training, validation, and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
from sklearn.svm import SVC
import pandas as pd

# Define SVM classifier
svm = SVC(kernel='linear', probability=True)

# Extract features from the training set
X_train_features = []
for image in X_train:
    features = extract_features(image)
    X_train_features.append(features)
X_train_features = np.array(X_train_features)
# Create a dataframe from the feature matrix and target variable
df = pd.DataFrame(X_train_features)
df['target'] = y_train

# Drop rows with NaN values
df.dropna(inplace=True)

# Split the dataframe back into X and y arrays
X_train_features = df.drop('target', axis=1).values
y_train = df['target'].values
# Train the SVM classifier
svm.fit(X_train_features, y_train)

<ipython-input-8-34a3e8ad271d>:57: RuntimeWarning: invalid value encountered in true_divide
  normalized_blocks[i, j] = block / np.linalg.norm(block, ord=1)


SVC(kernel='linear', probability=True)

In [11]:
# Extract features from the test set
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
X_test_features = []
for image in X_test:
    features = extract_features(image)
    X_test_features.append(features)
X_test_features = np.array(X_test_features)



# Create a dataframe from the feature matrix and target variable
df = pd.DataFrame(X_test_features)
df['target'] = y_test

# Drop rows with NaN values
df.dropna(inplace=True)

# Split the dataframe back into X and y arrays
X_test_features = df.drop('target', axis=1).values
y_test = df['target'].values

# Evaluate the SVM classifier on the test set
accuracy = svm.score(X_test_features, y_test)
y_pred = svm.predict(X_test_features)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display the evaluation metrics
print('Test Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

<ipython-input-8-34a3e8ad271d>:57: RuntimeWarning: invalid value encountered in true_divide
  normalized_blocks[i, j] = block / np.linalg.norm(block, ord=1)


Test Accuracy: 0.6428571428571429
Precision: 0.5471698113207547
Recall: 0.58
F1 Score: 0.5631067961165048


In [5]:
# Define function to predict the label and probability of a single image
def predict_single_image(image):
    # Extract HOG features from the image
    features = extract_features(image)
    # Reshape the features to match the shape of the training data
    features = features.reshape(1, -1)
    # Predict the label and probability using the trained SVM classifier
    label = svm.predict(features)[0]
    probability = svm.predict_proba(features)[0][label]
    return label, probability

# Load the single test image
image_path = 'test/b1.jpg'
image = cv2.imread(image_path)

# Predict the label and probability of the test image
if image is not None:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label, probability = predict_single_image(image)
    if (label==0):
      print('benign')
    else:
      print('malignant')
    print('Probability:', probability)

Label: 0
Probability: 0.5850666926548698
